## P&L Explain

 Daily P&L is computed as:
* Risk-based method: using greeks measured at SOD, multiply by observed market changes
* Step re-evaluation: bump each factor one at a time and reprice

In [1]:
import numpy as np
from scipy.stats import norm
import pandas as pd

# Pricing and Greeks

def bs_call_price(S, K, r, sigma, T):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

def bs_delta(S, K, r, sigma, T):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    return norm.cdf(d1)

def bs_gamma(S, K, r, sigma, T):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    return norm.pdf(d1) / (S * sigma * np.sqrt(T))

def bs_vega(S, K, r, sigma, T):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    return S * norm.pdf(d1) * np.sqrt(T)

# Define SOD market conditions

S0     = 100      # Start-of-day spot
K      = 100      # Strike
sigma0 = 0.20     # Start-of-day volatility
r0     = 0.02     # Start-of-day interest rate
T      = 1        # Time to maturity (years)

price_SOD = bs_call_price(S0, K, r0, sigma0, T)
delta_SOD = bs_delta(S0, K, r0, sigma0, T)
gamma_SOD = bs_gamma(S0, K, r0, sigma0, T)
vega_SOD  = bs_vega(S0, K, r0, sigma0, T)

price_SOD, delta_SOD, gamma_SOD, vega_SOD


(np.float64(8.916037278572539),
 np.float64(0.579259709439103),
 np.float64(0.019552134698772795),
 np.float64(39.104269397545586))

In [2]:
# Simulated observed changes in market factors (COB vs SOD)
dS     = +1.5    # Spot moved up by 1.5
dsigma = +0.01   # Volatility moved +1%
dr     = -0.001  # Rates moved -0.1%

S1     = S0 + dS
sigma1 = sigma0 + dsigma
r1     = r0 + dr

price_COB = bs_call_price(S1, K, r1, sigma1, T)
actual_PnL = price_COB - price_SOD
actual_PnL


np.float64(1.2294704681114297)

# Risk-based P&L (delta, gamma, vega approx.), corr. to the taylor expansion

In [5]:
PnL_delta = delta_SOD * dS
PnL_gamma = 0.5 * gamma_SOD * dS**2
PnL_vega  = vega_SOD  * dsigma

risk_based_PnL = PnL_delta + PnL_gamma + PnL_vega
risk_based_PnL


np.float64(1.28192840967023)

In [6]:
unexplained_risk_based = actual_PnL - risk_based_PnL
unexplained_risk_based

np.float64(-0.052457941558800236)

# Re-evaluation P&L (path independent), bumps 1 factor at a time, repriced from SOD and measured P&L

In [7]:
def price_with(S=S0, sigma=sigma0, r=r0):
    return bs_call_price(S, K, r, sigma, T)

# 1. Spot step: apply S -> S1, keep sigma0, r0
SR_spot = price_with(S=S1, sigma=sigma0, r=r0) - price_SOD

# 2. Vol step: apply sigma -> sigma1, keeping new spot S1
SR_vol  = price_with(S=S1, sigma=sigma1, r=r0) - price_with(S=S1, sigma=sigma0, r=r0)

# 3. Rate step: apply r -> r1, keeping new S1, sigma1
SR_r    = price_with(S=S1, sigma=sigma1, r=r1) - price_with(S=S1, sigma=sigma1, r=r0)

step_revaluation_PnL = SR_spot + SR_vol + SR_r
step_revaluation_PnL


np.float64(1.2294704681114297)

In [8]:
unexplained_step = actual_PnL - step_revaluation_PnL
unexplained_step


np.float64(0.0)

In [9]:
pd.DataFrame({
    "Actual P&L": [actual_PnL],
    "Risk-based P&L": [risk_based_PnL],
    "Risk-based unexplained": [unexplained_risk_based],
    "Step-revaluation P&L": [step_revaluation_PnL],
    "Step-revaluation unexplained": [unexplained_step]
})


,Actual P&L,Risk-based P&L,Risk-based unexplained,Step-revaluation P&L,Step-revaluation unexplained
0,1.22947,1.281928,-0.052458,1.22947,0.0


# Interpretation
1. Risk-based method
* uses delta, vega and is fast, suitable for intraday
* approx error grows with bigger shocks
2. Step-revaluation method
* unexplained_step should be nearly zero because BS is deterministic
